<a href="https://colab.research.google.com/github/tthandi/magic_formula_project/blob/main/MarketCap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
alpha_vantage_market_cap_full.py

• Computes market-cap snapshots for ALL tickers and ALL balance-sheet dates
  where Alpha Vantage reports commonStockSharesOutstanding (annual + quarterly).
• Matches each fiscal date to the most recent trading day ≤ that date using
  TIME_SERIES_DAILY_ADJUSTED (adjusted close).
• Robust:
  - API key via env var ALPHAVANTAGE_API_KEY (required)
  - Async + rate-limited (CALLS_PER_MIN=75 for premium)
  - Success-only checkpoint + failure log (Colab Drive if available)
  - Safe CSV loads (mkstemp, fixed column order), partitioned by yyyymm
  - Per-partition flush locks (avoid racey double-flush)
  - End-of-run de-dup per partition on (symbol, snapshotDate)

Modes are implicit: this script processes the AV universe minus checkpointed
symbols; re-runs will resume automatically.

Env:
  ALPHAVANTAGE_API_KEY (required)
  GOOGLE_APPLICATION_CREDENTIALS (optional, service account JSON for BigQuery)
"""

# ───────────────────────────── 1) CONFIG ─────────────────────────────
PROJECT   = ""
DATASET   = ""
TABLE     = "market_cap"

import os as _os
API_KEY   = '' # REQUIRED

CALLS_PER_MIN = 75       # premium
CONCURRENCY   = 32       # parallel symbols (two AV calls each)
FLUSH_ROWS    = 5_000
AUTO_SAVE_SEC = 300

CK_DIR_NAME   = "alpha_vantage_ckpt"
CK_FILE_NAME  = "av_done_market_cap2.txt"
FAIL_FILE     = "av_failed_market_cap2.txt"

PART_START, PART_END = 198001, 204001  # yyyymm

# ───────────────────────────── 2) IMPORTS ────────────────────────────
import os as _os
import io, csv, json, time, asyncio, tempfile, aiohttp, aiolimiter, nest_asyncio, argparse, datetime as dt
from collections import defaultdict
from typing import Dict, List, Set

import pandas as pd
import requests
import tqdm
from aiolimiter import AsyncLimiter

from google.cloud import bigquery
from google.api_core import exceptions as gex
from google.auth.exceptions import DefaultCredentialsError, RefreshError
from google.oauth2 import service_account

nest_asyncio.apply()

# ───────────────── 3) BigQuery client (auth-robust) ─────────────────
def get_bq_client() -> bigquery.Client:
    # Colab user auth
    try:
        from google.colab import auth as colab_auth  # type: ignore
        colab_auth.authenticate_user()
        return bigquery.Client(project=PROJECT)
    except Exception:
        pass
    # Service account
    sa_path = _os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
    if sa_path and _os.path.exists(sa_path):
        creds = service_account.Credentials.from_service_account_file(sa_path)
        return bigquery.Client(project=PROJECT, credentials=creds)
    # ADC (gcloud)
    try:
        return bigquery.Client(project=PROJECT)
    except (DefaultCredentialsError, RefreshError) as e:
        raise RuntimeError(
            "No Google Cloud credentials found. Use Colab auth, a service account via "
            "GOOGLE_APPLICATION_CREDENTIALS, or `gcloud auth application-default login`."
        ) from e

bq = get_bq_client()

# ──────────────── 4) Colab Drive checkpoint (if available) ─────────
def colab_drive_paths():
    try:
        from google.colab import drive  # type: ignore
        drive.mount("/content/drive", force_remount=False)
        root = "/content/drive/MyDrive"
        _os.makedirs(_os.path.join(root, CK_DIR_NAME), exist_ok=True)
        ck = _os.path.join(root, CK_DIR_NAME, CK_FILE_NAME)
        ff = _os.path.join(root, CK_DIR_NAME, FAIL_FILE)
        return ck, ff
    except Exception:
        return CK_FILE_NAME, FAIL_FILE

CHECKFILE, FAILFILE = colab_drive_paths()
_tqdmw = tqdm.tqdm.write
_tqdmw(f"✅ Checkpoint → {CHECKFILE}")

_ck_handle = open(CHECKFILE, "a", buffering=1)
_ck_lock   = asyncio.Lock()

async def mark_done(sym: str):
    async with _ck_lock:
        _ck_handle.write(sym + "\n"); _ck_handle.flush()
        _os.fsync(_ck_handle.fileno())

def read_checkpoint() -> Set[str]:
    try:    return set(open(CHECKFILE).read().splitlines())
    except FileNotFoundError: return set()

def mark_failure(sym: str):
    with open(FAILFILE, "a") as f:
        f.write(sym + "\n")

# ──────────────── 5) Alpha Vantage helpers (async) ─────────────────
BASE_URL     = "https://www.alphavantage.co/query"
RATE_LIMITER = AsyncLimiter(CALLS_PER_MIN, 60)

def date_to_yyyymm(d: dt.date) -> int: return d.year * 100 + d.month

async def av_get(session: aiohttp.ClientSession, **params):
    """Generic AV GET; returns JSON dict or raw text depending on response."""
    params |= {"apikey": API_KEY}
    async with RATE_LIMITER:
        try:
            async with session.get(BASE_URL, params=params, timeout=45) as r:
                if r.status != 200: return None
                ct = r.headers.get("Content-Type","")
                if "json" in ct:
                    return await r.json(content_type=None)
                return await r.text()
        except (aiohttp.ClientError, asyncio.TimeoutError):
            return None

async def listing(session: aiohttp.ClientSession, state: str, max_retries: int = 6) -> List[str]:
    """Robust LISTING_STATUS (CSV) with backoff; returns list of stock symbols."""
    params = {"function":"LISTING_STATUS","state":state,"datatype":"csv","apikey":API_KEY}
    backoff = 2
    for attempt in range(1, max_retries+1):
        async with RATE_LIMITER:
            try:
                async with session.get(BASE_URL, params=params, timeout=45) as resp:
                    status = resp.status
                    txt = await resp.text()
            except (aiohttp.ClientError, asyncio.TimeoutError):
                _tqdmw(f"⚠️ LISTING_STATUS {state}: network error (try {attempt})")
                await asyncio.sleep(backoff); backoff *= 2; continue
        if txt.lstrip().startswith("{"):
            try: j = json.loads(txt)
            except json.JSONDecodeError: j = {}
            msg = j.get("Note") or j.get("Information") or j.get("Error Message") or f"HTTP {status}"
            _tqdmw(f"⚠️ LISTING_STATUS {state}: {msg} (try {attempt})")
            await asyncio.sleep(backoff); backoff = min(backoff*2, 30); continue
        rdr = csv.DictReader(io.StringIO(txt))
        syms = [r["symbol"] for r in rdr if r.get("assetType")=="Stock" and r.get("symbol")]
        if syms: return syms
        _tqdmw(f"⚠️ LISTING_STATUS {state}: empty CSV (try {attempt})")
        await asyncio.sleep(backoff); backoff *= 2
    _tqdmw(f"❗ LISTING_STATUS {state}: giving up after {max_retries} attempts")
    return []

async def fetch_balance_sheets(session: aiohttp.ClientSession, sym: str) -> pd.DataFrame | None:
    """Return DataFrame of ALL (annual + quarterly) balance-sheet snapshots with shares."""
    data = await av_get(session, function="BALANCE_SHEET", symbol=sym)
    if not data: return None
    reports = (data.get("annualReports", []) + data.get("quarterlyReports", []))
    if not reports: return None
    df = pd.DataFrame(reports)
    if "commonStockSharesOutstanding" not in df.columns: return None
    df = df.replace({"None": pd.NA, "null": pd.NA, "": pd.NA})
    df = df[df["commonStockSharesOutstanding"].notna()]
    if df.empty: return None

    df["symbol"]           = sym
    df["fiscalDateEnding"] = pd.to_datetime(df["fiscalDateEnding"]).dt.date
    df["reportedCurrency"] = df.get("reportedCurrency", pd.NA)
    df["commonStockSharesOutstanding"] = pd.to_numeric(df["commonStockSharesOutstanding"], errors="coerce")
    df["yyyymm"]           = df["fiscalDateEnding"].map(date_to_yyyymm)

    return df[["symbol","fiscalDateEnding","reportedCurrency","commonStockSharesOutstanding","yyyymm"]]

async def fetch_price_series(session: aiohttp.ClientSession, sym: str, retries=1) -> pd.Series | None:
    """
    Return a Series of adjusted_close indexed by DATE (normalize to midnight).
    Retries once on JSON throttle.
    """
    txt = await av_get(session,
                       function="TIME_SERIES_DAILY_ADJUSTED",
                       symbol=sym,
                       outputsize="full",
                       datatype="csv")

    # Throttle/error returns JSON dict
    if isinstance(txt, dict):
        if retries:
            await asyncio.sleep(60)
            return await fetch_price_series(session, sym, retries-1)
        return None

    if not txt or txt.lstrip().startswith("{"):
        return None

    df = pd.read_csv(io.StringIO(txt), parse_dates=["timestamp"])
    # Normalize index to date (remove time) so lookups by dt.date work reliably
    df["date"] = df["timestamp"].dt.date
    df = df.set_index("date")
    return df["adjusted_close"]

def price_on_or_before(series: pd.Series, target_date: dt.date) -> tuple[dt.date, float] | None:
    """Get last available (date, price) where date <= target_date."""
    if series is None or series.empty: return None
    # Use index slice up to target_date, then take the last row if any.
    try:
        s = series.loc[:target_date]
        if s.size == 0: return None
        d = s.index[-1]
        return d, float(s.iloc[-1])
    except Exception:
        return None

# ───────────────── 6) BigQuery init (partitioned table) ─────────────
try: bq.get_dataset(f"{PROJECT}.{DATASET}")
except gex.NotFound: bq.create_dataset(bigquery.Dataset(f"{PROJECT}.{DATASET}"))

schema = [
    bigquery.SchemaField("symbol","STRING"),
    bigquery.SchemaField("snapshotDate","DATE"),
    bigquery.SchemaField("reportedCurrency","STRING"),
    bigquery.SchemaField("commonStockSharesOutstanding","FLOAT64"),
    bigquery.SchemaField("adjustedClose","FLOAT64"),
    bigquery.SchemaField("marketCap","FLOAT64"),
    bigquery.SchemaField("yyyymm","INT64"),
]
table_ref = f"{PROJECT}.{DATASET}.{TABLE}"
if not any(t.table_id == TABLE for t in bq.list_tables(DATASET)):
    tbl = bigquery.Table(table_ref, schema=schema)
    tbl.range_partitioning = bigquery.RangePartitioning(
        field="yyyymm", range_=bigquery.PartitionRange(PART_START, PART_END, 1))
    tbl.clustering_fields = ["symbol"]
    bq.create_table(tbl)
    _tqdmw(f"➡️  Created table {table_ref}")

CSV_COLS = [f.name for f in bq.get_table(table_ref).schema]

def load_to_bq(df: pd.DataFrame, yyyymm: int, retries: int = 4):
    """Append rows to the specific partition (table$yyyymm)."""
    if df is None or df.empty: return
    df = df.reindex(columns=CSV_COLS)
    fd, tmp = tempfile.mkstemp(suffix=".csv.gz"); _os.close(fd)
    df.to_csv(tmp, index=False, compression="gzip", date_format="%Y-%m-%d")

    dest = f"{table_ref}${yyyymm}"
    cfg = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        write_disposition="WRITE_APPEND",
    )
    backoff = 5
    for attempt in range(1, retries+1):
        try:
            with open(tmp, "rb") as fh:
                job = bq.load_table_from_file(fh, dest, job_config=cfg, rewind=True)
            _tqdmw(f"⬆️  load {dest} rows {len(df)} (try {attempt})")
            job.result(timeout=300)
            _tqdmw(f"✅ partition {yyyymm} rows {len(df)}")
            break
        except (gex.GoogleAPICallError, gex.RetryError, TimeoutError) as e:
            _tqdmw(f"⚠️  {dest} {e} — backoff {backoff}s")
            time.sleep(backoff); backoff = min(backoff*2, 60)
    _os.remove(tmp)

def dedupe_partitions(parts: Set[int]):
    """De-dup within each yyyymm on (symbol, snapshotDate)."""
    if not parts:
        _tqdmw("🧹 dedupe: no partitions to process")
        return

    sql = f"""
    -- Loop through each partition p
    FOR p IN (SELECT part FROM UNNEST(@parts) AS part) DO

      -- Build cleaned partition
      CREATE OR REPLACE TEMP TABLE tmp AS
      SELECT * EXCEPT(rn)
      FROM (
        SELECT
          t.*,
          ROW_NUMBER() OVER (
            PARTITION BY symbol, snapshotDate
            ORDER BY yyyymm DESC
          ) AS rn
        FROM `{table_ref}` AS t
        WHERE yyyymm = p.part          -- ✅ use p.part here
      )
      WHERE rn = 1;

      -- Replace existing rows for that partition
      DELETE FROM `{table_ref}` WHERE yyyymm = p.part;  -- ✅ and here
      INSERT INTO `{table_ref}` SELECT * FROM tmp;

    END FOR;
    """

    job = bq.query(
        sql,
        job_config=bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ArrayQueryParameter("parts", "INT64", sorted(parts))
            ]
        ),
    )
    job.result()
    _tqdmw(f"🧹 dedupe: processed {len(parts)} partition(s)")



# ─────────────── 7) Pending buffers + autosave (locked) ────────────
_pending: Dict[int, List[pd.DataFrame]] = defaultdict(list)
_pending_rows: Dict[int, int]           = defaultdict(int)
_part_locks: Dict[int, asyncio.Lock]    = {}

async def flush_partition(part: int):
    lock = _part_locks.setdefault(part, asyncio.Lock())
    async with lock:
        if not _pending[part]:
            return
        batch = pd.concat(_pending[part])
        _pending[part].clear(); _pending_rows[part] = 0
        await asyncio.to_thread(load_to_bq, batch, part)

async def flush_all(tag="💾 flush"):
    tasks = [flush_partition(p) for p,v in list(_pending.items()) if v]
    if tasks:
        _tqdmw(f"{tag}: {len(tasks)} partitions")
        await asyncio.gather(*tasks)

async def auto_flush():
    while True:
        await asyncio.sleep(AUTO_SAVE_SEC)
        await flush_all("⏳ autosave")

# ─────────────────────────── 8) Main driver ────────────────────────
async def main():
    if not API_KEY:
        raise SystemExit("❌  Set ALPHAVANTAGE_API_KEY and retry.")

    done = read_checkpoint()

    connector = aiohttp.TCPConnector(limit=CONCURRENCY)
    async with aiohttp.ClientSession(connector=connector,
                                     timeout=aiohttp.ClientTimeout(total=90)) as session:
        active, delisted = await asyncio.gather(
            listing(session, "active"),
            listing(session, "delisted")
        )
        _tqdmw(f"ℹ️ LISTING_STATUS → active: {len(active):,}, delisted: {len(delisted):,}")
        universe = sorted(set(active + delisted))
        if not universe:
            _tqdmw("❗ LISTING_STATUS returned 0; aborting (premium key expected to work).")
            return

        todo = [s for s in universe if s not in done]
        _tqdmw(f"Universe {len(universe):,} | remaining {len(todo):,}")

        sem  = asyncio.Semaphore(CONCURRENCY)
        pbar = tqdm.tqdm(total=len(todo), desc="Tickers processed", dynamic_ncols=True)
        saver = asyncio.create_task(auto_flush())
        touched_parts: Set[int] = set()

        async def worker(sym: str):
            # Two AV calls per symbol; success-only checkpoint
            async with sem:
                try:
                    bs = await fetch_balance_sheets(session, sym)
                    if bs is None or bs.empty:
                        mark_failure(sym); pbar.update(1); return

                    prices = await fetch_price_series(session, sym)
                    if prices is None or prices.empty:
                        mark_failure(sym); pbar.update(1); return

                    rows = []
                    for _, r in bs.iterrows():
                        report_date: dt.date = r["fiscalDateEnding"]
                        price_hit = price_on_or_before(prices, report_date)
                        if not price_hit:
                            continue
                        snap_date, price = price_hit
                        shares = float(r["commonStockSharesOutstanding"])
                        if pd.isna(shares) or pd.isna(price):  # skip bad rows
                            continue
                        rows.append({
                            "symbol": sym,
                            "snapshotDate": snap_date,
                            "reportedCurrency": r["reportedCurrency"],
                            "commonStockSharesOutstanding": shares,
                            "adjustedClose": float(price),
                            "marketCap": float(price) * float(shares),
                            "yyyymm": date_to_yyyymm(snap_date),
                        })

                    if rows:
                        df = pd.DataFrame(rows).reindex(columns=CSV_COLS)
                        for part, grp in df.groupby("yyyymm"):
                            _pending[part].append(grp)
                            _pending_rows[part] += len(grp)
                            touched_parts.add(int(part))
                            # thresholded flush with lock
                            if _pending_rows[part] >= FLUSH_ROWS:
                                await flush_partition(part)
                        await mark_done(sym)
                    else:
                        mark_failure(sym)
                finally:
                    pbar.update(1)

        await asyncio.gather(*(asyncio.create_task(worker(s)) for s in todo))
        await flush_all("✅ final")
        saver.cancel()
        pbar.close()

    # End-of-run de-dup only for partitions we touched
    await asyncio.to_thread(dedupe_partitions, touched_parts)

# ─────────────────────────── 9) Entry point ────────────────────────
if __name__ == "__main__":
    # Jupyter/Colab friendly: ignore unknown argv
    parser = argparse.ArgumentParser()
    args, _unknown = parser.parse_known_args()
    asyncio.run(main())
